###Importing

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn import preprocessing, model_selection, neighbors
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
import os
!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE


from sklearn import preprocessing, model_selection, neighbors
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
import os
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans

##1. Preprocessing and Feature Engineering


> Add blockquote




In [3]:
df = pd.read_csv("creditcard.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'creditcard.csv'

In [ ]:
df

In [ ]:
data=df.copy()
data.head()

In [ ]:
df.fillna(data.mean(), inplace=True)

In [ ]:
df

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
X = df.drop('Class', axis=1)
y = df['Class']

In [ ]:
y.value_counts()

In [ ]:
y.astype(int)

In [ ]:
df['Class'] = df['Class'].astype(int)

In [ ]:
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")

In [ ]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:

# Now proceed with SMOTE
# Set k_neighbors to a value less than or equal to the smallest minority class size
smote = SMOTE(random_state=42, k_neighbors=3) # or even lower like 2, or 1
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
print(f"After SMOTE: \n{y_train_resampled.value_counts()}")

In [ ]:
X_train=X_resampled
y_train=y_resampled

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##Import test dataset

In [ ]:
import os
print(os.listdir("/content/sample_data/"))

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
uploaded_df = pd.DataFrame([uploaded])

In [ ]:
scaler = StandardScaler()
# Fit the StandardScaler on the training data or suitable data
scaler.fit(X_train_resampled)  # Assuming X_train is your training data

# Now you can transform the uploaded data
uploaded_df_scaled = scaler.transform(uploaded_df)

In [ ]:
import os
print(os.listdir("/content/sample_data/"))

from google.colab import files
uploaded = files.upload()



In [ ]:
# Assuming you know the correct file name after uploading
uploaded_df = pd.read_csv("Classification_Test_file (1).csv")

# Ensure columns match the training data
uploaded_df = uploaded_df[X_train.columns]  # Reorder columns if needed

scaler = StandardScaler()
# Fit the StandardScaler on the training data or suitable data
scaler.fit(X_train_resampled)  # Assuming X_train is your training data

# Now you can transform the uploaded data
uploaded_df_scaled = scaler.transform(uploaded_df)

In [ ]:
y_pred_DT = dt.predict(uploaded_df_scaled)


##SVM

In [ ]:

y_train.shape

(566506,)

In [ ]:
X_train_scaled.shape

(566506, 30)

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
# Hyperparameter tuning using GridSearchCV for SVM
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
svm = SVC(random_state=42)
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')


NameError: name 'y_train_scaled' is not defined

In [ ]:
grid_search.fit(X_train_scaled, y_train)
# Get the best SVM model from GridSearchCV
best_svm = grid_search.best_estimator_

In [ ]:

# Make predictions using the best SVM model
y_pred_svm = best_svm.predict(x)
submission_svm = pd.DataFrame({
    'ID': range(len(y_pred_svm)),
    'Prediction': y_pred_svm
})
# Save the predictions to a CSV file
submission_svm.to_csv('submission_SVM_Shahd.csv', index=False)

##HyperParameter Tuning
>GridSearchCV

>RandomizedSearchCV

>Bayesian Optimization



###Grid SearchVC

####Decision Tree

In [ ]:
# Hyperparameter Tuning with Grid Search for Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

dt = DecisionTreeClassifier(random_state=42)
grid_search_dt = GridSearchCV(estimator=dt, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search_dt.fit(X_train_scaled, y_train)

print("Best Parameters for Decision Tree:", grid_search_dt.best_params_)
print("Best Score:", grid_search_dt.best_score_)


In [ ]:

# Evaluate the tuned Decision Tree on the test set
best_dt = grid_search_dt.best_estimator_

# Instead of using X_original (which was incorrect), use the original X data for prediction
y_pred_DT_GS = best_dt.predict(uploaded_df_scaled)

# from sklearn.metrics import accuracy_score, classification_report

# accuracy = accuracy_score(y, y_pred_DT_GS)
# print("Decision Tree Accuracy:", accuracy)
# print("Classification Report:")
# print(classification_report(y, y_pred_DT_GS))

# Save predictions to CSV
import pandas as pd
submission = pd.DataFrame({
    'ID': range(len(y_pred_DT_GS)),
    'Prediction': y_pred_DT_GS
})
submission.to_csv('submission_DT_GS.csv', index=False)

In [ ]:
# Hyperparameter Tuning with Grid Search for Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

rf = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search_rf.fit(X, y)

print("Best Parameters for Random Forest:", grid_search_rf.best_params_)
print("Best Score:", grid_search_rf.best_score_)


####RandomForest

In [ ]:
# Evaluate the tuned Random Forest on the test set
best_rf = grid_search_rf.best_estimator_
y_pred_RF = best_rf.predict(uploaded_df_scaled)

from sklearn.metrics import accuracy_score, classification_report

# accuracy = accuracy_score(y, y_pred_RF)
# print("Random Forest Accuracy:", accuracy)
# print("Classification Report:")
# print(classification_report(y, y_pred_RF))

# Save predictions to CSV
import pandas as pd

submission = pd.DataFrame({
    'ID': range(len(y_pred_RF)),
    'Prediction': y_pred_RF
})
submission.to_csv('submission_RF_GS.csv', index=False)


NameError: name 'grid_search_rf' is not defined

####NN

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

# Reduced hyperparameter grid
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50)],  # Removed larger configurations
    'activation': ['relu', 'tanh'],  # Fewer activation functions
    'solver': ['adam'],  # Focusing on the most commonly used solver
    'learning_rate': ['constant', 'adaptive'],  # Keep only necessary options
    'max_iter': [200]  # Use a smaller iteration range for faster tuning
}

mlp = MLPClassifier(random_state=42)

# Reduce cross-validation folds and enable parallel processing
grid_search_nn = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

# Fit the model
grid_search_nn.fit(X_train_resampled, y_train)

# Display results
print("Best Parameters for Neural Network:", grid_search_nn.best_params_)
print("Best Score:", grid_search_nn.best_score_)


In [ ]:
# Evaluate the tuned Neural Network on the test set
best_nn = grid_search_nn.best_estimator_
y_pred_NN = best_nn.predict(uploaded_df_scaled)

from sklearn.metrics import accuracy_score, classification_report

# accuracy = accuracy_score(y, y_pred_NN)
# print("Neural Network Accuracy:", accuracy)
# print("Classification Report:")
# print(classification_report(y, y_pred_NN))

# Save predictions to CSV
import pandas as pd

submission = pd.DataFrame({
    'ID': range(len(y_pred_NN)),
    'Prediction': y_pred_NN
})
submission.to_csv('submission_NN_GS.csv', index=False)


###RandomizedSearchCV

####DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Simplified hyperparameter space
param_distributions = {
    'criterion': ['gini', 'entropy'],  # Keep both options
    'max_depth': [None] + list(np.arange(10, 50, 10)),  # Wider steps
    'min_samples_split': np.arange(2, 11, 2),  # Narrowed range
    'min_samples_leaf': np.arange(1, 11, 2),  # Narrowed range
    'max_features': [None, 'sqrt']  # Removed 'log2' to reduce combinations
}

dt = DecisionTreeClassifier(random_state=42)

# RandomizedSearchCV with reduced iterations and folds
random_search_dt = RandomizedSearchCV(
    estimator=dt,
    param_distributions=param_distributions,
    n_iter=30,  # Reduced number of combinations
    scoring='accuracy',
    cv=3,  # Reduced cross-validation folds
    random_state=42,
    verbose=2,
    n_jobs=-1  # Enable parallel processing
)

# Fit RandomizedSearchCV
random_search_dt.fit(X_train_scaled, y_train)

print("Best Parameters for Decision Tree:", random_search_dt.best_params_)
print("Best Score:", random_search_dt.best_score_)


In [ ]:
# Evaluate the tuned Decision Tree on the test set
best_dt = random_search_dt.best_estimator_
y_pred_DT = best_dt.predict(uploaded_df_scaled)

from sklearn.metrics import accuracy_score, classification_report

# accuracy = accuracy_score(y, y_pred_DT)
# print("Decision Tree Accuracy:", accuracy)
# print("Classification Report:")
# print(classification_report(y, y_pred_DT))

# Save predictions to CSV
import pandas as pd

submission = pd.DataFrame({
    'ID': range(len(y_pred_DT)),
    'Prediction': y_pred_DT
})
submission.to_csv('submission_DT_RS.csv', index=False)


####RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Simplified hyperparameter space
param_distributions = {
    'n_estimators': np.arange(50, 301, 50),  # Reduced range for faster evaluation
    'max_depth': [None] + list(np.arange(20, 101, 20)),  # Wider steps
    'min_samples_split': np.arange(2, 11, 2),  # Reduced range
    'min_samples_leaf': np.arange(1, 11, 2),  # Reduced range
    'criterion': ['gini'],  # Focus on one to simplify
    'max_features': ['sqrt', 'log2']  # Reduced options
}

rf = RandomForestClassifier(random_state=42)

# RandomizedSearchCV with reduced iterations and folds
random_search_rf = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=30,  # Reduced number of iterations
    scoring='accuracy',
    cv=3,  # Reduced cross-validation folds
    random_state=42,
    verbose=2,
    n_jobs=-1  # Enable parallel processing
)

# Fit RandomizedSearchCV
random_search_rf.fit(X_train_scaled, y_train)

print("Best Parameters for Random Forest:", random_search_rf.best_params_)
print("Best Score:", random_search_rf.best_score_)


In [ ]:
# Evaluate the tuned Random Forest on the test set
best_rf = random_search_rf.best_estimator_
y_pred_RF = best_rf.predict(uploaded_df_scaled)

from sklearn.metrics import accuracy_score, classification_report

# accuracy = accuracy_score(y, y_pred_RF)
# print("Random Forest Accuracy:", accuracy)
# print("Classification Report:")
# print(classification_report(y, y_pred_RF))

# Save predictions to CSV
import pandas as pd

submission = pd.DataFrame({
    'ID': range(len(y_pred_RF)),
    'Prediction': y_pred_RF
})
submission.to_csv('submission_RF_RS.csv', index=False)


####NN

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Simplified hyperparameter space
param_distributions = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],  # Reduced configurations
    'activation': ['relu', 'tanh'],  # Focus on effective activation functions
    'solver': ['adam'],  # Adam works well and is computationally efficient
    'alpha': np.logspace(-3, 0, 4),  # Narrowed regularization parameter range
    'learning_rate': ['constant', 'adaptive'],  # Removed 'invscaling'
    'max_iter': [200, 300]  # Limited iterations for faster training
}

mlp = MLPClassifier(random_state=42)

# RandomizedSearchCV with reduced iterations and folds
random_search_nn = RandomizedSearchCV(
    estimator=mlp,
    param_distributions=param_distributions,
    n_iter=20,  # Reduced number of random combinations
    scoring='accuracy',
    cv=3,  # Reduced cross-validation folds
    random_state=42,
    verbose=2,
    n_jobs=-1  # Enable parallel processing
)

# Fit RandomizedSearchCV
random_search_nn.fit(X_train_scaled, y_train)

print("Best Parameters for Neural Network:", random_search_nn.best_params_)
print("Best Score:", random_search_nn.best_score_)


In [ ]:
# Evaluate the tuned Neural Network on the test set
best_nn = random_search_nn.best_estimator_
y_pred_NN = best_nn.predict(uploaded_df_scaled)

# from sklearn.metrics import accuracy_score, classification_report

# accuracy = accuracy_score(y, y_pred_NN)
# print("Neural Network Accuracy:", accuracy)
# print("Classification Report:")
# print(classification_report(y, y_pred_NN))

# Save predictions to CSV
import pandas as pd

submission = pd.DataFrame({
    'ID': range(len(y_pred_NN)),
    'Prediction': y_pred_NN
})
submission.to_csv('submission_NN_RS.csv', index=False)


###Bayesian Optimization

####DecisionTree

In [ ]:
!pip install scikit-optimize==0.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install skopt

ERROR: Could not find a version that satisfies the requirement skopt (from versions: none)
ERROR: No matching distribution found for skopt


In [ ]:
# Bayesian Optimization for Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV

# Define the search space
search_space = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None] + list(range(1, 21)),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20)
}

dt = DecisionTreeClassifier(random_state=42)

# Instantiate BayesSearchCV
bayes_search_dt = BayesSearchCV(
    estimator=dt,
    search_spaces=search_space,
    n_iter=50,  # Number of parameter settings that are sampled
    cv=5,  # Number of folds in cross-validation
    scoring='accuracy',
    random_state=42,
    verbose=2
)

# Fit BayesSearchCV
bayes_search_dt.fit(X, y)

print("Best Parameters for Decision Tree:", bayes_search_dt.best_params_)
print("Best Score:", bayes_search_dt.best_score_)


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# Evaluate the tuned Decision Tree on the test set
best_dt = bayes_search_dt.best_estimator_
y_pred_DT = best_dt.predict(X)

from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y, y_pred_DT)
print("Decision Tree Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y, y_pred_DT))

# Save predictions to CSV
import pandas as pd

submission = pd.DataFrame({
    'ID': range(len(y_pred_DT)),
    'Prediction': y_pred_DT
})
submission.to_csv('submission_DT_BC.csv', index=False)


####RandomForest

In [ ]:
# Bayesian Optimization for Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
!pip install scikit-optimize
from skopt import BayesSearchCV
import numpy as np

# Define the search space
search_space = {
    'n_estimators': (50, 500),  # Number of trees in the forest
    'max_depth': [None] + list(range(5, 30)),  # Maximum depth of the tree
    'min_samples_split': (2, 20),  # Minimum number of samples to split
    'min_samples_leaf': (1, 20),  # Minimum number of samples at a leaf node
    'criterion': ['gini', 'entropy'],  # Splitting criteria
    'max_features': ['sqrt', 'log2']  # Number of features to consider, removed 'auto'
}

rf = RandomForestClassifier(random_state=42)

# Instantiate BayesSearchCV
bayes_search_rf = BayesSearchCV(
    estimator=rf,
    search_spaces=search_space,
    n_iter=50,  # Number of parameter settings that are sampled
    cv=5,  # Number of folds in cross-validation
    scoring='accuracy',
    random_state=42,
    verbose=2
)

# This is the workaround to avoid the error.  Change np.int to int
np.int = int

# Fit BayesSearchCV
bayes_search_rf.fit(X_train_scaled, y_train)

print("Best Parameters for Random Forest:", bayes_search_rf.best_params_)
print("Best Score:", bayes_search_rf.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END criterion=entropy, max_depth=22, max_features=sqrt, min_samples_leaf=7, min_samples_split=14, n_estimators=236; total time=12.6min
[CV] END criterion=entropy, max_depth=22, max_features=sqrt, min_samples_leaf=7, min_samples_split=14, n_estimators=236; total time=13.3min
[CV] END criterion=entropy, max_depth=22, max_features=sqrt, min_samples_leaf=7, min_samples_split=14, n_estimators=236; total time=12.8min
[CV] END criterion=entropy, max_depth=22, max_features=sqrt, min_samples_leaf=7, min_samples_split=14, n_estimators=236; total time=13.0min
[CV] END criterion=entropy, max_depth=22, max_features=sqrt, min_samples_leaf=7, min_samples_split=14, n_estimators=236; total time=13.4min
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END criterion=gini, max_depth=26, max_features=log2, min_samples_leaf=19, min_samples_split=18, n_estimators=78; total time= 3.9min
[CV] END criterion=gini, max_depth=26, max_fe

KeyboardInterrupt: 

In [ ]:

t=pd.read_csv("Classification_Test_Data.csv")


# Evaluate the tuned Random Forest on the test set
best_rf = bayes_search_rf.best_estimator_
y_pred_RF = best_rf.predict(t)

from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y, y_pred_RF)
print("Random Forest Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y, y_pred_RF))

# Save predictions to CSV
import pandas as pd

submission = pd.DataFrame({
    'ID': range(len(y_pred_RF)),
    'Prediction': y_pred_RF
})
submission.to_csv('submission_RF_BC.csv', index=False)


####NN

In [ ]:
from sklearn.neural_network import MLPClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

# Define the search space
search_space = {
    'hidden_layer_sizes': Categorical([(50, 50), (100, 100), (100, 50), (200, 100)]),  # List of explicit tuples for hidden layers
    'activation': Categorical(['relu', 'tanh', 'logistic']),  # Categorical for activation function
    'solver': Categorical(['adam', 'sgd']),  # Categorical for solver choice
    'alpha': Real(1e-5, 1e-1, prior='log-uniform'),  # Regularization term
    'learning_rate': Categorical(['constant', 'invscaling', 'adaptive']),  # Categorical for learning rate
    'max_iter': Integer(200, 1000)  # Max iterations for training
}

mlp = MLPClassifier(random_state=42)

# Instantiate BayesSearchCV
bayes_search_nn = BayesSearchCV(
    estimator=mlp,
    search_spaces=search_space,
    n_iter=50,
    cv=5,
    scoring='accuracy',
    random_state=42,
    verbose=2
)

# Fit BayesSearchCV
bayes_search_nn.fit(X, y)

# Output the best parameters and score
print("Best Parameters for Neural Network:", bayes_search_nn.best_params_)
print("Best Score:", bayes_search_nn.best_score_)


In [ ]:
# Evaluate the tuned Neural Network on the test set
best_nn = bayes_search_nn.best_estimator_
y_pred_NN = best_nn.predict(X)

from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y, y_pred_NN)
print("Neural Network Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y, y_pred_NN))

# Save predictions to CSV
import pandas as pd

submission = pd.DataFrame({
    'ID': range(len(y_pred_NN)),
    'Prediction': y_pred_NN
})
submission.to_csv('submission_NN_BC.csv', index=False)


##VOTING

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Replace `best_dt`, `best_rf`, `best_nn` with the tuned models from GridSearchCV
voting_clf_grid = VotingClassifier(estimators=[
    ('DT_Grid', grid_search_dt.best_estimator_),
    ('RF_Grid', grid_search_rf.best_estimator_),
    ('NN_Grid', grid_search_nn.best_estimator_)
], voting='hard')  # Use 'soft' for probabilities

# Fit the VotingClassifier for GridSearchCV results
voting_clf_grid.fit(X, y)
y_pred_grid = voting_clf_grid.predict(X)

# Evaluate the results
print("Voting Classifier with GridSearchCV:")
print("Accuracy:", accuracy_score(y, y_pred_grid))
print("Classification Report:")
print(classification_report(y, y_pred_grid))

# Save GridSearchCV predictions
submission_grid = pd.DataFrame({
    'ID': range(len(y_pred_grid[:994])),  # Keep only the first 994 rows
    'Prediction': y_pred_grid[:994]
})
submission_grid.to_csv('submission_Voting_Grid.csv', index=False)

# Replace `best_dt`, `best_rf`, `best_nn` with the tuned models from RandomizedSearchCV
voting_clf_random = VotingClassifier(estimators=[
    ('DT_Random', random_search_dt.best_estimator_),
    ('RF_Random', random_search_rf.best_estimator_),
    ('NN_Random', random_search_nn.best_estimator_)
], voting='hard')  # Use 'soft' for probabilities

# Fit the VotingClassifier for RandomizedSearchCV results
voting_clf_random.fit(X, y)
y_pred_random = voting_clf_random.predict(X)

# Evaluate the results
print("\nVoting Classifier with RandomizedSearchCV:")
print("Accuracy:", accuracy_score(y, y_pred_random))
print("Classification Report:")
print(classification_report(y, y_pred_random))

# Save RandomizedSearchCV predictions
submission_random = pd.DataFrame({
    'ID': range(len(y_pred_random[:994])),  # Keep only the first 994 rows
    'Prediction': y_pred_random[:994]
})
submission_random.to_csv('submission_Voting_Random.csv', index=False)

# Replace `best_dt`, `best_rf`, `best_nn` with the tuned models from Bayesian Optimization
voting_clf_bayes = VotingClassifier(estimators=[
    ('DT_Bayes', bayes_search_dt.best_estimator_),
    ('RF_Bayes', bayes_search_rf.best_estimator_),
  #  ('NN_Bayes', bayes_search_nn.best_estimator_)
], voting='hard')  # Use 'soft' for probabilities

# Fit the VotingClassifier for Bayesian Optimization results
voting_clf_bayes.fit(X, y)
y_pred_bayes = voting_clf_bayes.predict(X)

# Evaluate the results
print("\nVoting Classifier with Bayesian Optimization:")
print("Accuracy:", accuracy_score(y, y_pred_bayes))
print("Classification Report:")
print(classification_report(y, y_pred_bayes))

# Save Bayesian Optimization predictions
submission_bayes = pd.DataFrame({
    'ID': range(len(y_pred_bayes[:994])),  # Keep only the first 994 rows
    'Prediction': y_pred_bayes[:994]
})
submission_bayes.to_csv('submission_Voting_Bayes.csv', index=False)


#MODELS


##DecisionTree

In [ ]:
# Instead of using X_train_resampled and y_train,
# use X_resampled and y_resampled, which were generated correctly:
dt = DecisionTreeClassifier(random_state=5)
dt.fit(X_resampled, y_resampled)  # Use the correctly resampled data


NameError: name 't' is not defined

In [ ]:
import os
print(os.listdir("/content/sample_data/"))

['anscombe.json', 'README.md', 'mnist_train_small.csv', 'mnist_test.csv', 'california_housing_train.csv', 'california_housing_test.csv']


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Classification_Test_file.csv to Classification_Test_file (1).csv


In [ ]:
uploaded_df = pd.DataFrame([uploaded])

In [ ]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()
import os
print(os.listdir())

Saving Classification_Test_file.csv to Classification_Test_file (3).csv
['.config', '.ipynb_checkpoints', 'Classification_Test_file11.csv', 'Classification_Test_file.csv', 'Classification_Test_file (2).csv', 'Classification_Test_file (1).csv', 'creditcard.csv', 'Classification_Test_file (3).csv', 'sample_data']


In [ ]:


# Assuming you know the correct file name after uploading
uploaded_df = pd.read_csv("Classification_Test_file (3).csv")

# Ensure columns match the training data
uploaded_df = uploaded_df[X_train.columns]  # Reorder columns if needed

# If the model used scaling, apply the same transformation
uploaded_df_scaled = scaler.transform(uploaded_df)

# Predict
y_pred_DT = dt.predict(uploaded_df_scaled)




/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [ ]:
accuracy = dt.score(X, y)
print("DT Accuracy:",accuracy)
print(classification_report(y, y_pred_DT))

In [ ]:
import pandas as pd

submission = pd.DataFrame({
    'ID': range(len(y_pred_DT)),
    'Prediction': y_pred_DT
})

submission.to_csv('submission_DT.csv', index=False)


##RandomForest
low accuracy

In [46]:
rf = RandomForestClassifier(random_state = 5)
rf.fit(X_train_resampled, y_train_resampled)
y_pred_RF = rf.predict(uploaded_df_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
cm = confusion_matrix(y, y_pred_RF)
cm

In [ ]:
precision_score(y, y_pred_RF)

In [ ]:
accuracy = rf.score(X, y)
print("RF Accuracy:",accuracy)
print(classification_report(y, y_pred_RF))

In [47]:
import pandas as pd  # Keep only the first 994 predictions

submission = pd.DataFrame({
    'ID': range(len(y_pred_RF)),
    'Prediction': y_pred_RF
})

submission.to_csv('submission_RF.csv', index=False)


##NN

In [41]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neural_network import MLPClassifier


In [42]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=500, random_state=42)

mlp.fit(X_train_scaled, y_train)



MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)

In [43]:
y_pred_NN = mlp.predict(uploaded_df_scaled)



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [45]:
# accuracy = accuracy_score(y, y_pred_NN)
# print(f"Accuracy: {accuracy:.2f}")
# print("Classification Report:")
# print(classification_report(y, y_pred_NN))


ValueError: Found input variables with inconsistent numbers of samples: [283726, 994]

In [44]:
import pandas as pd
y_pred = y_pred_NN  # Keep only the first 994 predictions

submission = pd.DataFrame({
    'ID': range(len(y_pred)),
    'Prediction': y_pred
})

submission.to_csv('submission_NN.csv', index=False)


##Voting

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import mean_squared_error
# Create a Voting Regressor
voting_reg = VotingClassifier(estimators=[
    ('DT', dt), ('rf', rf), ('mlp', mlp)
])

# Train and evaluate the voting regressor
voting_reg.fit(X, y)
y_pred_Voting = voting_reg.predict(X)
mse_voting = mean_squared_error(y, y_pred)
print(f"Voting Regressor MSE: {mse_voting:.4f}")


In [ ]:
import pandas as pd
y_pred = y_pred_Voting[:994]  # Keep only the first 994 predictions

submission = pd.DataFrame({
    'ID': range(len(y_pred)),
    'Prediction': y_pred
})

submission.to_csv('submission_Voting.csv', index=False)
